In [56]:
from lxml import etree
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import tushare as ts
import tqdm
pro = ts.pro_api('5e2da4e6322b3b893f1c10e43d001749300351741d736d247251d837')

In [59]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

with open('龙虎榜营业部.csv','a',encoding='gbk') as f:
    f.write('date,stock,营业部,买入(万),卖出(万),净买(万)\n')

df2 = pro.trade_cal(exchange='', start_date='20180101', end_date='20211202')
cal = df2[df2['is_open']==1]['cal_date'].tolist()
cal_format = list(map(lambda x:'{}-{}-{}'.format(x[:4],x[4:6],x[-2:]),cal))

for date in tqdm.tqdm_notebook(cal_format):
    with open('龙虎榜营业部.csv','a',encoding='gbk') as f:
        url = 'http://data.10jqka.com.cn/ifmarket/lhbggxq/report/%s/'%date
        html = requests.get(url=url,headers=headers)
        if html.status_code==200:
            soup = BeautifulSoup(html.text, 'lxml')
            elements = soup.find_all(attrs={'class':'stockcont'})
            for element in elements:
                tmp_text = element.find('p').text
                name,tmp_text2 = tmp_text.split(')')
                a = list(map(lambda x:x.text.replace('\n',''),element.find_all('td')))
                b = np.array(a).reshape(-1,4)
                length = b.shape[0]
                df = pd.DataFrame(b,index=[[date]*length,[name]*length],columns=['营业部','买入','卖出','净买'])
                df.index.names = ['date','stock']
                string = '\r\n'.join(df.to_csv().split('\r\n')[1:]).replace('\r','').replace('一线游资','')
                f.write(string)
        else:
            pring('%s凉了'%date)

<ipython-input-59-000f36fcfb39>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for date in tqdm.tqdm_notebook(cal_format):


In [ ]:
#daily update 由于csv追加写入，不能乱运行。
date = input('yyyy-mm-dd:')
with open('龙虎榜营业部.csv','a',encoding='gbk') as f:
    url = 'http://data.10jqka.com.cn/ifmarket/lhbggxq/report/%s/'%date
    html = requests.get(url=url,headers=headers)
    if html.status_code==200:
        soup = BeautifulSoup(html.text, 'lxml')
        elements = soup.find_all(attrs={'class':'stockcont'})
        for element in elements:
            tmp_text = element.find('p').text
            name,tmp_text2 = tmp_text.split(')')
            a = list(map(lambda x:x.text.replace('\n',''),element.find_all('td')))
            b = np.array(a).reshape(-1,4)
            length = b.shape[0]
            df = pd.DataFrame(b,index=[[date]*length,[name]*length],columns=['营业部','买入','卖出','净买'])
            df.index.names = ['date','stock']
            string = '\r\n'.join(df.to_csv().split('\r\n')[1:]).replace('\r','').replace('一线游资','')
            f.write(string)
    else:
        pring('%s凉了'%date)

In [62]:
df = pd.read_csv('龙虎榜营业部.csv',index_col=[0,1],encoding='gbk')
df

营业部   买入(万)    卖出(万)    净买(万)
date       stock                                                         
2018-01-02 金岭矿业(000655  国泰君安证券股份有限公司成都北一环路证券...  612.19     0.00   612.19
           金岭矿业(000655   国泰君安证券股份有限公司北京方庄路证券营业部  328.52     0.00   328.52
           金岭矿业(000655    国信证券股份有限公司深圳泰然九路证券营业部  305.82   162.13   143.69
           金岭矿业(000655  国金证券股份有限公司上海奉贤区金碧路证券...  197.17    14.60   182.56
           金岭矿业(000655   国泰君安证券股份有限公司泸州星光路证券营业部  160.67     0.00   160.67
...                                         ...     ...      ...      ...
2021-12-01 煜邦电力(688597   东方财富证券股份有限公司上海东方路证券营业部    0.00  1080.57 -1080.57
           煜邦电力(688597        中国国际金融股份有限公司上海分公司    0.00  1047.90 -1047.90
           煜邦电力(688597   申万宏源西部证券有限公司济南历山路证券营业部    0.00   513.27  -513.27
           煜邦电力(688597     华泰证券股份有限公司常州东横街证券营业部    0.00   501.13  -501.13
           煜邦电力(688597        华泰证券股份有限公司总部一线游资     0.00   498.93  -498.93

[580922 rows x 4 columns]

In [75]:
df_2021 = df.iloc[list(map(lambda x:x[0][:4]=='2021', df.index)),:]
df_2021.groupby('营业部').sum().sort_values('买入(万)',ascending=False ).head(53).tail(50)

,买入(万),卖出(万),净买(万)
营业部,,,
东方财富证券股份有限公司拉萨团结路第二证...一线游资,7446005.78,6438490.21,1007515.20
华鑫证券有限责任公司上海分公司一线游资,7083672.87,4511999.62,2571673.32
中国国际金融股份有限公司上海黄浦区湖滨路...,4310591.05,2959226.51,1351364.51
华泰证券股份有限公司总部一线游资,3520713.67,2514274.71,1006438.96
招商证券交易单元(353800),3077145.65,3184129.19,-106983.60
东方财富证券股份有限公司拉萨东环路第一证...,2912663.66,2336820.84,575842.90
东方财富证券股份有限公司拉萨团结路第一证...,2447622.20,1969341.74,478280.19
国泰君安证券股份有限公司上海江苏路证券营业部,2005768.07,1180251.99,825516.03
国泰君安证券股份有限公司南京太平南路证券...,1901370.05,854548.88,1046821.16
